In [1]:
import os

In [3]:
from snowflake.snowpark.session import Session

In [2]:
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

In [4]:
connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

[Row(status='Statement executed successfully.')]

In [8]:
tables = []
for row in session.sql("show tables;").collect():
    if row.name.startswith("PEEK"):
        tables.append(row.name)
tables

['PEEK11_202964b6_e615_4d1f_a961_9833dff2f4bd_259523',
 'PEEK12_202964b6_e615_4d1f_a961_9833dff2f4bd_259523',
 'PEEK13_202964b6_e615_4d1f_a961_9833dff2f4bd_259523',
 'PEEK2_202964b6_e615_4d1f_a961_9833dff2f4bd_259523',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_259538',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_265718',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_271987',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_271997',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272094',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272101',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272110',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272121',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272132',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272139',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272148',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272155',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272164',
 'PEEK2_621e407a_9330_4fad_a179_5967afc64c83_272171',
 'PEEK2_621e407a_9330_4fa

In [11]:
for table in tables:
    session.sql('DROP TABLE IF EXISTS {};'.format(table)).collect()